## RAG Demo Using Offline Docs

### Import the packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("env_vars.env")) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

### Load the documents

In [2]:
from langchain.document_loaders import PyPDFLoader

loaders = [PyPDFLoader("../docs/Inview_June_2023.pdfInview_June_2023.pdf")]
docs = []
for loader in loaders:
    docs.extend(loader.load())
    
print(len(docs))

6


In [3]:
# Split into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)
print(len(splits))

24


### Store "chunks" as vectors

In [6]:
# # The embedding libraries
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from langchain.embeddings import SentenceTransformerEmbeddings

# from langchain.vectorstores import Chroma

# persist_directory = '../docs/chroma-test/'
# !rm -rf ../docs/chroma-test/  # remove old database files if any

# # # OpenAI embedding
# # embeddings = OpenAIEmbeddings()

# # # Try a different embedding (HF)
# # embeddings = HuggingFaceEmbeddings()

# # Try a different embedding (ST)
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
# # Store in the local vector DB
# vectordb = Chroma.from_documents(
#     documents=splits,
#     embedding=embeddings,
#     persist_directory=persist_directory
# )

# print(vectordb._collection.count())

### Semantic search

In [ ]:
question = "What industry sectors are recommended for stocks?"

docs = vectordb.similarity_search(question, k=3)
print(len(docs))

In [ ]:
# Inspect the top-ranked result
docs[0].page_content

In [ ]:
for d in docs:
    print(d.metadata)

In [16]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + 
                                   d.page_content for i, d in enumerate(docs)]))

In [ ]:
pretty_print_docs(docs)

In [ ]:
# Persist the vector DB for RAG
vectordb.persist()

### Retrieval

In [8]:
from langchain.chat_models import ChatOpenAI

# Invoke the LLM
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/Users/trucvietle/miniforge3/envs/llm-env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# Create QA chain and prompt template
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Build the prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run the chain
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [17]:
# question = "What is the main topic of this document?"
# question = "What industry sectors are recommended for stocks?"
question = "What is the house view between stocks and bonds?"

# Print out the retrieved contexts
retrieved_docs = retriever.get_relevant_documents(question)
pretty_print_docs(retrieved_docs)

# result = qa_chain({"query": question})
# result["result"]

Document 1:

including local currency emerging market debt. Finally, 
to limit the riskiness of the portfolio, exposure to high 
yield bonds should be reduced in favour of investment 
grade corporate bonds.
03 Global and Fixed Income Asset Allocation 04 Equity Sector Views
04 Equities Asset Allocation 05 Alternatives Asset AllocationContents
----------------------------------------------------------------------------------------------------
Document 2:

that inflation is declining globally. This is partly thanks 
to the fall in the prices of raw materials. Furthermore, 
services prices are also starting to return to levels 
more consistent with central banks’ targets. The 
tightening of monetary policy also appears to be 
nearing completion in many parts of the world and 
this is supportive for markets.At the same time, the earnings season has been better 
than expected, especially for technology companies. 
The brighter prospects for corporate profitability in 
2023 are another favora

In [26]:
contexts = [d.page_content for d in retrieved_docs]

prompt_start = (
    "Use the following pieces of context to answer the question at the end.\
    If you don't know the answer, just say that you don't know, don't try to make up an answer.\
    Use three sentences maximum. Keep the answer as concise as possible.\n\n"+
    "Context:\n"
)

prompt_end = (
    f"\n\nQuestion: {question}\nAnswer:"
)

prompt = (
    prompt_start + "\n\n---\n\n".join(contexts) + 
    prompt_end
)

print(prompt)

Use the following pieces of context to answer the question at the end.    If you don't know the answer, just say that you don't know, don't try to make up an answer.    Use three sentences maximum. Keep the answer as concise as possible.

Context:
including local currency emerging market debt. Finally, 
to limit the riskiness of the portfolio, exposure to high 
yield bonds should be reduced in favour of investment 
grade corporate bonds.
03 Global and Fixed Income Asset Allocation 04 Equity Sector Views
04 Equities Asset Allocation 05 Alternatives Asset AllocationContents

---

that inflation is declining globally. This is partly thanks 
to the fall in the prices of raw materials. Furthermore, 
services prices are also starting to return to levels 
more consistent with central banks’ targets. The 
tightening of monetary policy also appears to be 
nearing completion in many parts of the world and 
this is supportive for markets.At the same time, the earnings season has been better 
than

In [29]:
# Try on local models
from llama_cpp import Llama

# model_path = "/Users/trucvietle/Downloads/llm-models/mistral-7b-instruct-v0.1.Q6_K.gguf"
model_path = "/Users/trucvietle/Downloads/llm-models/llama-2-7b-chat.Q4_K_M.gguf"

llm = Llama(model_path=model_path,
            n_ctx=8192, n_batch=512,
            n_threads=7, n_gpu_layers=2,
            verbose=False, seed=42)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/trucvietle/Downloads/llm-models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.a

In [30]:
output = llm(prompt, echo=True, stream=False, max_tokens=4096)

output_str = output["choices"][0]["text"].replace(prompt, "")
print(output_str)

 The house view is that a moderate overweight in both equities and bonds remains appropriate. However, some adjustments in the allocation within asset classes are advisable.


In [ ]:
# result["source_documents"][2]

In [ ]:
question = "What does the document suggest about currencies?"

result = qa_chain({"query": question})
result["result"]

In [ ]:
# pretty_print_docs(result["source_documents"])

### Alternative: Compression retrieval

In [34]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Wrap our vectorstore
# Use local model
from langchain.llms import LlamaCpp

model_path = "/Users/trucvietle/Downloads/llm-models/llama-2-7b-chat.Q4_K_M.gguf"
model = LlamaCpp(model_path=model_path,
                 n_ctx=8192, n_batch=512,
                 n_threads=7, n_gpu_layers=2)
compressor = LLMChainExtractor.from_llm(model)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_kwargs={"k": 3})
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/trucvietle/Downloads/llm-models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.a

In [35]:
question = "What does the document suggest about bonds?"

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)


llama_print_timings:        load time =    8970.05 ms
llama_print_timings:      sample time =      11.11 ms /    87 runs   (    0.13 ms per token,  7832.19 tokens per second)
llama_print_timings: prompt eval time =    8969.67 ms /   179 tokens (   50.11 ms per token,    19.96 tokens per second)
llama_print_timings:        eval time =  193519.65 ms /    86 runs   ( 2250.23 ms per token,     0.44 tokens per second)
llama_print_timings:       total time =  202932.78 ms /   265 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8970.05 ms
llama_print_timings:      sample time =      10.86 ms /    77 runs   (    0.14 ms per token,  7093.51 tokens per second)
llama_print_timings: prompt eval time =   10599.29 ms /   459 tokens (   23.09 ms per token,    43.30 tokens per second)
llama_print_timings:        eval time =  212124.74 ms /    76 runs   ( 2791.11 ms per token,     0.36 tokens per second)
llama_print_timings:       total time =  223182.30 ms /   535 

Document 1:

> "Finally, to limit the riskiness of the portfolio, exposure to high yield bonds should be reduced in favour of investment grade corporate bonds."
This part of the context suggests that the document is discussing the allocation of fixed income assets within a portfolio and provides guidance on reducing exposure to high-yield bonds in favor of investment-grade corporate bonds.
----------------------------------------------------------------------------------------------------
Document 2:

* Bonds (mentioned multiple times)
* Federal Reserve (mentioned)
* Interest rates (mentioned)
* Inflation (mentioned)
* Economy (mentioned)
* Credit (mentioned)
* Spreads (mentioned)
* Duration (mentioned)
* Currency (mentioned)
----------------------------------------------------------------------------------------------------
Document 3:

* Bonds
* Bond yields
* Monetary policy
* Asset allocation
* Equities
* Fixed income assets



llama_print_timings:        load time =    8970.05 ms
llama_print_timings:      sample time =       4.83 ms /    30 runs   (    0.16 ms per token,  6215.04 tokens per second)
llama_print_timings: prompt eval time =   10268.66 ms /   368 tokens (   27.90 ms per token,    35.84 tokens per second)
llama_print_timings:        eval time =   75285.93 ms /    29 runs   ( 2596.07 ms per token,     0.39 tokens per second)
llama_print_timings:       total time =   85736.55 ms /   397 tokens


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "What does the document suggest about bonds?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
# Try on Mistral 7B model (offline)
from llama_cpp import Llama

model_path = "/Users/trucvietle/Downloads/llm-models/mistral-7b-instruct-v0.1.Q6_K.gguf"
llm = Llama(model_path=model_path,
            n_ctx=8192, n_batch=512,
            n_threads=7, n_gpu_layers=2,
            verbose=False, seed=42)

In [36]:
query = "What does the document suggest about bonds?"

contexts = [d.page_content for d in compressed_docs]

prompt_start = (
    "Answer the question based on the context below.\n\n"+
    "Context:\n"
)

prompt_end = (
    f"\n\nQuestion: {query}\nAnswer:"
)

prompt = (
    prompt_start + "\n\n---\n\n".join(contexts) + 
    prompt_end
)

print(prompt)

Answer the question based on the context below.

Context:
> "Finally, to limit the riskiness of the portfolio, exposure to high yield bonds should be reduced in favour of investment grade corporate bonds."
This part of the context suggests that the document is discussing the allocation of fixed income assets within a portfolio and provides guidance on reducing exposure to high-yield bonds in favor of investment-grade corporate bonds.

---

* Bonds (mentioned multiple times)
* Federal Reserve (mentioned)
* Interest rates (mentioned)
* Inflation (mentioned)
* Economy (mentioned)
* Credit (mentioned)
* Spreads (mentioned)
* Duration (mentioned)
* Currency (mentioned)

---

* Bonds
* Bond yields
* Monetary policy
* Asset allocation
* Equities
* Fixed income assets

Question: What does the document suggest about bonds?
Answer:


In [37]:
output = llm(prompt, echo=True, stream=False, max_tokens=4096)

output_str = output["choices"][0]["text"].replace(prompt, "")
print(output_str)

 The document suggests that exposure to high-yield bonds should be reduced in favor of investment-grade corporate bonds, indicating that bonds are an important component of the portfolio and their risk is a concern.


## Create a chatbot!!

In [ ]:
import panel as pn  # GUI
pn.extension()

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [ ]:
# from langchain.chat_models import ChatOpenAI

# # Invoke the LLM
# llm_name = "gpt-3.5-turbo"
# llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [ ]:
def load_db(file, chain_type, k):
    
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    
    # define embedding
    embeddings = OpenAIEmbeddings()
    
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    
    return qa 

In [ ]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "../docs/Inview_June_2023.pdfInview_June_2023.pdf"
        self.qa = load_db(self.loaded_file, "stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
            
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [ ]:
# Create a chatbot
cb = cbfs()

In [ ]:
# Create the panels
file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)

In [ ]:
dashboard